In [8]:
from google.colab import userdata
api_key=userdata.get('GOOGLE_API_KEY')

In [2]:
!pip install google-genai

In [12]:
import google.generativeai as genai
from PIL import Image

genai.configure(api_key=api_key)

model = genai.GenerativeModel("gemini-2.0-flash")

In [13]:
#for Answersheet

In [15]:
prompt_answersheet = f"""
Analyse precisely and do not hellusinate.Give question number and their Marks obtained in json format.consider that Question numbsr only which has Obtained marks handwritten,.Do not wite null or anything just laeve that .Give output in json format
ignore total marks section
"""

In [32]:
def get_response(image_path, prompt):
  image = Image.open(image_path)

  # Generate a textual response from the image
  response = model.generate_content([image, prompt])

  # Print the description
  return response.text

In [20]:
image_path = "/content/answer_sheet_1.jpg"
Answersheet=get_response(image_path, prompt_answersheet)

In [21]:
print(Answersheet)

```json
{
    "1": "10",
    "2": "2",
    "3": "3",
    "4": "0"
}
```


In [ ]:
#for Question Paper

In [40]:
prompt_Question = f"""
Analyse the Given question Paper.Give questions and theirs max marks in json format if a  question has more than one subpart then give  question as their subpart but max maarks of combined below that whole subpart.Give output in json format
"""

In [41]:
image_path_qn = "/content/test_paper_1.jpg"
Question_Paper=get_response(image_path_qn, prompt_Question)

In [42]:
print(Question_Paper)

```json
{
    "questions": [
        {
            "Q1a": "RTL is used to describe...",
            "Q1b": "Sign of the number is preserved in ... shift micro-operation.",
            "Q1c": "PC contains the address of ...",
            "Q1d": "What decisions are made in computer organization?",
            "Q1e": "Convert (568)10->()2->()->()16",
            "Q1f": "Define micro-operation and cite an example of micro-operation.",
            "Q1g": "Differentiate fixed point and floating point representation.",
            "Q1h": "Why extra memory reference is needed in indirect memory address instruction?",
            "Q1i": "Write the instruction to load and store accumulator register?",
            "Q1j": "How I bit classify the type of instruction?",
            "max_marks": 15
        },
        {
            "Q2": "With the help of flow diagram discuss instruction execution cycle.",
            "max_marks": 5
        },
        {
            "Q3": "Present a report on the use o

In [43]:
#Combining Both of two

In [44]:
#few sort prompting for model
outputs="""

{
  "questions": [
    {
      "question_number": "Q1",
      "description": "Instruction execution cycle",
      "max_marks": 5,
      "marks_obtained": 2
    }
  ]
}
"""


prompt_combining_ocr=f"""
  You are given two JSON objects: {Answersheet}` and :{Question_Paper}.

- {Question_Paper} contains a list of questions, subparts, descriptions, and maximum marks.
- {Answersheet}contains the marks obtained for each question.

Your task is to merge these two JSONs and give output only json object.

Return a **single valid JSON object** with all questions, subparts, `"max_marks"`, and `"marks_obtained"`.

yout output will be like {outputs} .you will give just output not code


"""

In [49]:
combined_res= model.generate_content(prompt_combining_ocr).text

In [50]:
print(combined_res)

```json
{
  "questions": [
    {
      "Q1a": "RTL is used to describe...",
      "Q1b": "Sign of the number is preserved in ... shift micro-operation.",
      "Q1c": "PC contains the address of ...",
      "Q1d": "What decisions are made in computer organization?",
      "Q1e": "Convert (568)10->()2->()->()16",
      "Q1f": "Define micro-operation and cite an example of micro-operation.",
      "Q1g": "Differentiate fixed point and floating point representation.",
      "Q1h": "Why extra memory reference is needed in indirect memory address instruction?",
      "Q1i": "Write the instruction to load and store accumulator register?",
      "Q1j": "How I bit classify the type of instruction?",
      "max_marks": 15,
      "marks_obtained": 10
    },
    {
      "Q2": "With the help of flow diagram discuss instruction execution cycle.",
      "max_marks": 5,
      "marks_obtained": 2
    },
    {
      "Q3": "Present a report on the use of common bus system for data transfer? Illustrate d

In [ ]:
#fnal answer

In [51]:
prompt_final=f"just tell me the topic and araange them according to obtained marks in ascending order considering this {combined_res}"

In [52]:
final= model.generate_content(prompt_final).text

In [53]:
print(final)

Here's the topic list, ordered by ascending marks obtained:

1.  **Q4**: Explain memory reference and register reference instructions with suitable examples. (Marks: 0)
2.  **Q2**: With the help of flow diagram discuss instruction execution cycle. (Marks: 2)
3.  **Q3**: Present a report on the use of common bus system for data transfer? Illustrate data transfer using common bus system through example. (Marks: 3)
4.  **Q1a-Q1j**: Various short answer questions covering RTL, shift operations, PC content, computer organization decisions, number conversions, micro-operations, fixed vs floating point, indirect addressing, load/store instructions, instruction type classification. (Marks: 10)

